In [1]:
import os
os.chdir("/content/drive/MyDrive/AI/Data")

In [6]:
import pandas as pd
csv_data = pd.read_csv("titanic_train.csv")
csv_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
'''
'PassengerId': '승객 고유 식별자 (정수)',
    'Survived'   : '생존 여부 (0 = 사망, 1 = 생존)',
    'Pclass'     : '좌석 등급 (1=1등석, 2=2등석, 3=3등석)',
    'Name'       : '승객 이름 (문자열)',
    'Sex'        : '성별 (male/female)',
    'Age'        : '나이 (연속형 숫자)',
    'SibSp'      : '동반한 형제자매/배우자 수',
    'Parch'      : '동반한 부모/자녀 수',
    'Ticket'     : '티켓 번호 (문자열)',
    'Fare'       : '운임 요금 (연속형 숫자)',
    'Cabin'      : '객실 번호 (문자열, 결측치 多)',
    'Embarked'   : '탑승 항구 (C = Cherbourg, Q = Queenstown, S = Southampton)'
'''

In [7]:
# 불필요한 열 리스트
cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']

# 열 제거
csv_data.drop(columns=cols_to_drop, inplace=True)

In [8]:
print("제거 후 남은 열:", csv_data.columns.tolist())

제거 후 남은 열: ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']


In [9]:
# 삭제 전 결측치 현황
print("결측치 개수 (행 기준):")
print(csv_data.isnull().sum())

# 결측치 포함 행 삭제
csv_data.dropna(inplace=True)

# 삭제 후 요약 정보
print("\n=== 전처리 후 데이터 정보 ===")
csv_data.info()

결측치 개수 (행 기준):
Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

=== 전처리 후 데이터 정보 ===
<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    int64  
 2   Sex       712 non-null    object 
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Fare      712 non-null    float64
 7   Embarked  712 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 50.1+ KB


In [10]:
# 1) 인코딩할 범주형 열 지정
cat_cols = ['Sex', 'Embarked']

# 2) 원-핫 인코딩 수행
#    - drop_first=True: 기준(dummy trap 방지)으로 첫 번째 카테고리 제거
csv_data_encoded = pd.get_dummies(csv_data, columns=cat_cols, drop_first=True)

# 3) 결과 확인
print(csv_data_encoded.columns.tolist())
csv_data_encoded.head()

['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S']


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,True,False,True
1,1,1,38.0,1,0,71.2833,False,False,False
2,1,3,26.0,0,0,7.9250,False,False,True
3,1,1,35.0,1,0,53.1000,False,False,True
4,0,3,35.0,0,0,8.0500,True,False,True


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# 예시: df_encoded에 'Survived' 타깃과 나머지 특성이 있다고 가정
titanic_data = csv_data_encoded.drop(columns=['Survived'])
titanic_target = csv_data_encoded['Survived']

# 80% 학습, 20% 테스트 분할 (random_state 고정)
train_data, test_data, train_target, test_target = train_test_split(
    titanic_data, titanic_target, test_size=0.2, random_state=42, stratify=titanic_target)

In [12]:
# 1) Pipeline 정의
pipe_knn = Pipeline([
    ('scaler', StandardScaler()),      # 수치형 스케일링
    ('knn', KNeighborsClassifier())
])

# 2) 탐색할 하이퍼파라미터 그리드
param_grid_knn = {
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__weights'    : ['uniform', 'distance'],
    'knn__p'          : [1, 2]        # p=1 맨해튼, p=2 유클리드
}

# 3) GridSearchCV 객체 생성
grid_knn = GridSearchCV(
    estimator=pipe_knn,
    param_grid=param_grid_knn,
    cv=5,                # 5-폴드 교차검증
    scoring='accuracy',
    n_jobs=-1            # 병렬 처리
)

# 4) 학습 및 최적 파라미터 확인
grid_knn.fit(train_data, train_target)
print("KNN Best Params:", grid_knn.best_params_)
print("KNN Best CV Acc:", grid_knn.best_score_)
print("KNN Test Acc :", grid_knn.score(test_data, test_target))

KNN Best Params: {'knn__n_neighbors': 7, 'knn__p': 1, 'knn__weights': 'uniform'}
KNN Best CV Acc: 0.8207576463282098
KNN Test Acc : 0.8041958041958042


In [13]:
# 1) Pipeline 정의
pipe_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 2) 탐색할 하이퍼파라미터 그리드
param_grid_svm = {
    'svm__C'     : [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf'],
    'svm__gamma' : ['scale', 'auto']
}

# 3) GridSearchCV 객체 생성
grid_svm = GridSearchCV(
    estimator=pipe_svm,
    param_grid=param_grid_svm,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# 4) 학습 및 결과 출력
grid_svm.fit(train_data, train_target)
print("SVM Best Params:", grid_svm.best_params_)
print("SVM Best CV Acc:", grid_svm.best_score_)
print("SVM Test Acc :", grid_svm.score(test_data, test_target))

SVM Best Params: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
SVM Best CV Acc: 0.8224499301350722
SVM Test Acc : 0.7832167832167832


In [14]:
# 1) Pipeline 정의 (RandomForest는 스케일링 영향이 적지만 일관성을 위해 포함)
pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=42))
])

# 2) 탐색할 하이퍼파라미터 그리드
param_grid_rf = {
    'rf__n_estimators'    : [100, 200],
    'rf__max_depth'       : [None, 10, 20],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf' : [1, 2]
}

# 3) GridSearchCV 객체 생성
grid_rf = GridSearchCV(
    estimator=pipe_rf,
    param_grid=param_grid_rf,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# 4) 학습 및 결과 출력
grid_rf.fit(train_data, train_target)
print("RF Best Params:", grid_rf.best_params_)
print("RF Best CV Acc:", grid_rf.best_score_)
print("RF Test Acc :", grid_rf.score(test_data, test_target))

RF Best Params: {'rf__max_depth': 10, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 200}
RF Best CV Acc: 0.8260208042229469
RF Test Acc : 0.7902097902097902


In [15]:
pred_data = pd.read_csv("titanic_test.csv")

# 2) 불필요한 열(PassengerId, Name, Ticket, Cabin) 제거
cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
pred_data.drop(columns=cols_to_drop, inplace=True)

# 3) 결측치가 있는 행 삭제
#    (훈련 시에도 Age, Fare, Embarked 결측치 행을 drop 했으므로 동일하게 처리)
pred_data.dropna(inplace=True)

# 4) 범주형 변수 Sex, Embarked → 원-핫 인코딩
#    drop_first=True 로 기준 카테고리 하나씩 제거(dummy trap 방지)
pred_data_encoded = pd.get_dummies(
    pred_data,
    columns=['Sex', 'Embarked'],
    drop_first=True
)

# 5) (선택) 훈련 데이터와 컬럼 순서·개수를 맞추기
#    train_data.columns 가 이미 정의되어 있다면,
#    누락된 컬럼은 0으로 채워주고 순서를 동일하게 맞춰야 모델에 입력 오류가 없습니다.
missing_cols = set(train_data.columns) - set(pred_data_encoded.columns)
for c in missing_cols:
    pred_data_encoded[c] = 0
# 컬럼 순서 맞추기
pred_data_encoded = pred_data_encoded[train_data.columns]

# 6) 결과 확인
print(pred_data_encoded.shape)
pred_data_encoded.head()

(331, 8)


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,True,True,False
1,3,47.0,1,0,7.0000,False,False,True
2,2,62.0,0,0,9.6875,True,True,False
3,3,27.0,0,0,8.6625,True,False,True
4,3,22.0,1,1,12.2875,False,False,True


In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1) 모델 리스트 정의
models = [
    ('KNN', grid_knn),
    ('SVM', grid_svm),
    ('RandomForest', grid_rf)
]

# 2) 각 모델별 최적 파라미터 출력 및 테스트 예측
for name, grid in models:
    print(f"=== {name} 모델 ===")

    # 최적 하이퍼파라미터
    print("최적 파라미터:", grid.best_params_)

    # 테스트 데이터 예측
    preds = grid.predict(test_data)

    # 정확도
    acc = accuracy_score(test_target, preds)
    print(f"테스트 정확도: {acc:.4f}")

    # 클래스별 정밀도/재현율 등 리포트
    print("분류 리포트:")
    print(classification_report(test_target, preds, digits=4))

    # 혼동 행렬
    cm = confusion_matrix(test_target, preds)
    print("혼동 행렬 (행: 실제, 열: 예측)")
    print(pd.DataFrame(
        cm,
        index=['실제 0 (사망)', '실제 1 (생존)'],
        columns=['예측 0 (사망)', '예측 1 (생존)']
    ))

    print("\n")

=== KNN 모델 ===
최적 파라미터: {'knn__n_neighbors': 7, 'knn__p': 1, 'knn__weights': 'uniform'}
테스트 정확도: 0.8042
분류 리포트:
              precision    recall  f1-score   support

           0     0.8353    0.8353    0.8353        85
           1     0.7586    0.7586    0.7586        58

    accuracy                         0.8042       143
   macro avg     0.7970    0.7970    0.7970       143
weighted avg     0.8042    0.8042    0.8042       143

혼동 행렬 (행: 실제, 열: 예측)
           예측 0 (사망)  예측 1 (생존)
실제 0 (사망)         71         14
실제 1 (생존)         14         44


=== SVM 모델 ===
최적 파라미터: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
테스트 정확도: 0.7832
분류 리포트:
              precision    recall  f1-score   support

           0     0.7935    0.8588    0.8249        85
           1     0.7647    0.6724    0.7156        58

    accuracy                         0.7832       143
   macro avg     0.7791    0.7656    0.7702       143
weighted avg     0.7818    0.7832    0.7805       143

혼동 행렬 (